<a href="https://colab.research.google.com/github/Shimonimehrotra/Automatic-Lung-Segmentation/blob/main/evaluate_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jun  6 12:25:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip '/content/gdrive/MyDrive/TARP_LUNG_SEG/tbh+clahe.zip' -d '/content'

Archive:  /content/gdrive/MyDrive/TARP_LUNG_SEG/tbh+clahe.zip
   creating: /content/tbh+clahe/
   creating: /content/tbh+clahe/test/
   creating: /content/tbh+clahe/test/cxrs/
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0048_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0005_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0013_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0061_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0338_1.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0070_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0083_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0047_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0080_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0266_1.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0024_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0101_0.png  
  inflating: /content/tbh+clahe/test/cxrs/MCUCXR_0071_0.png  
  inflating: /cont

In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 9.1MB/s 


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import backend
from tensorflow.keras.models import load_model
from glob import glob
import os
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Nadam


class FixedDropout(layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        symbolic_shape = backend.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)


smooth = 1e-6

#all the errors defined here,as in the training_model.ipynb
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)


def generalised_dice_loss_2d(Y_gt, Y_pred):
    w = tf.reduce_sum(Y_gt)
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator)
    numerator = tf.reduce_sum(numerator)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator)
    denominator = tf.reduce_sum(denominator)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss


def pgdl(y_true, y_pred):
    k = 2.5
    gd = generalised_dice_loss_2d(y_true, y_pred)
    loss = gd / (1 + k * (1 - gd))
    return loss


alpha = 0.9


def loss(y_true, y_pred):
    return alpha * pgdl(y_true, y_pred) + (1 - alpha) * binary_crossentropy(y_true, y_pred)


model = load_model('/content/gdrive/MyDrive/TARP_LUNG_SEG/net_lung_seg.hdf5',
                   custom_objects={"FixedDropout": FixedDropout, "loss": loss, "dice_coef": dice_coef, "pgdl": pgdl,
                                   "generalised_dice_loss_2d": generalised_dice_loss_2d, "jacard_coef": jacard_coef,
                                   "dice_coef_loss": dice_coef_loss})

model.compile(optimizer=Nadam(), loss=loss,
              metrics=[binary_crossentropy, dice_coef, jacard_coef, 'binary_accuracy'])

def val_generator(batch_size, train_path, image_folder, mask_folder,
                  image_color_mode="rgb",
                  mask_color_mode="grayscale",
                  target_size=(512, 512)):
    image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator()
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)

    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)

    for (img, mask) in zip(image_generator, mask_generator):
        img, mask = normalise_val(img, mask)
        yield img, mask


def normalise_val(img, mask):
    img = img / 255.0
    mask = mask / 255.0
    mask[mask > 0.5] = 1.0
    mask[mask <= 0.5] = 0.0
    return img, mask


BATCH_SIZE = 2
segmentation_train_path = '/content/tbh+clahe/train'
segmentation_validation_path = '/content/tbh+clahe/validation'
segmentation_test_path = '/content/tbh+clahe/test'
segmentation_validation_image_path = '/content/tbh+clahe/validation/cxrs/'
segmentation_train_image_path = '/content/tbh+clahe/train/cxrs/'
segmentation_test_image_path = '/content/tbh+clahe/test/cxrs/'
train_files = glob(os.path.join(segmentation_train_image_path, "*.png"))
val_files = glob(os.path.join(segmentation_validation_image_path, "*.png"))
test_files = glob(os.path.join(segmentation_test_image_path, "*.png"))

train_gen = val_generator(BATCH_SIZE,
                          segmentation_train_path,
                          'cxrs',
                          'masks',
                          target_size=(512, 512))

val_gen = val_generator(BATCH_SIZE,
                        segmentation_validation_path,
                        'cxrs',
                        'masks',
                        target_size=(512, 512))

test_gen = val_generator(BATCH_SIZE,
                         segmentation_test_path,
                         'cxrs',
                         'masks',
                         target_size=(512, 512))
model.evaluate(train_gen,
               batch_size=BATCH_SIZE,
               verbose=1,
               steps=len(train_files) / BATCH_SIZE)
model.evaluate(val_gen,
               batch_size=BATCH_SIZE,
               verbose=1,
               steps=len(val_files) / BATCH_SIZE)
model.evaluate(test_gen,
               batch_size=BATCH_SIZE,
               verbose=1,
               steps=len(test_files) / BATCH_SIZE)

# Plot Model
plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=100,
)

Found 307 images belonging to 1 classes.
Found 307 images belonging to 1 classes.
153/153 [==============================] - 56s 119ms/step - loss: 0.0245 - binary_crossentropy: 0.0187 - dice_coef: 0.9868 - jacard_coef: 0.9739 - binary_accuracy: 0.9933
Found 39 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
19/19 [==============================] - 2s 107ms/step - loss: 0.0287 - binary_crossentropy: 0.0427 - dice_coef: 0.9798 - jacard_coef: 0.9607 - binary_accuracy: 0.9895
Found 39 images belonging to 1 classes.
Found 39 images belonging to 1 classes.
19/19 [==============================] - 2s 107ms/step - loss: 0.0261 - binary_crossentropy: 0.0320 - dice_coef: 0.9824 - jacard_coef: 0.9654 - binary_accuracy: 0.9910
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.72201 to fit



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import backend
from tensorflow.keras.models import load_model
from glob import glob
import os
import tensorflow as tf
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2


class FixedDropout(layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        symbolic_shape = backend.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)


smooth = 1e-6


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + smooth)


def generalised_dice_loss_2d(Y_gt, Y_pred):
    w = tf.reduce_sum(Y_gt)
    w = 1 / (w ** 2 + smooth)

    numerator = Y_gt * Y_pred
    numerator = w * tf.reduce_sum(numerator)
    numerator = tf.reduce_sum(numerator)

    denominator = Y_pred + Y_gt
    denominator = w * tf.reduce_sum(denominator)
    denominator = tf.reduce_sum(denominator)

    gen_dice_coef = 2 * numerator / (denominator + smooth)
    loss = tf.reduce_mean(1 - gen_dice_coef)
    return loss


def pgdl(y_true, y_pred):
    k = 2.5
    gd = generalised_dice_loss_2d(y_true, y_pred)
    loss = gd / (1 + k * (1 - gd))
    return loss


alpha = 0.9


def loss(y_true, y_pred):
    return alpha * pgdl(y_true, y_pred) + (1 - alpha) * binary_crossentropy(y_true, y_pred)


model = load_model('/content/gdrive/MyDrive/TARP_LUNG_SEG/net_lung_seg.hdf5',
                   custom_objects={"FixedDropout": FixedDropout, "loss": loss, "dice_coef": dice_coef, "pgdl": pgdl,
                                   "generalised_dice_loss_2d": generalised_dice_loss_2d, "jacard_coef": jacard_coef,
                                   "dice_coef_loss": dice_coef_loss})

files = []


def cxr_generator(batch_size, image_path, image_folder, files,
                  image_color_mode="rgb",
                  target_size=(512, 512)):
    image_datagen = ImageDataGenerator()
    image_generator = image_datagen.flow_from_directory(
        image_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False)
    image_generator.reset()
    files.extend(image_generator.filenames)
    for img in image_generator:
        img = img / 255.0
        yield img


BATCH_SIZE = 1
segmentation_train_path = '/content/tbh+clahe/train'
segmentation_validation_path = '/content/tbh+clahe/validation'
segmentation_test_path = '/content/tbh+clahe/test'
segmentation_validation_image_path = '/content/tbh+clahe/validation/cxrs/'
segmentation_train_image_path = '/content/tbh+clahe/train/cxrs/'
segmentation_test_image_path = '/content/tbh+clahe/test/cxrs/'
train_files = glob(os.path.join(segmentation_train_image_path, "*.png"))
val_files = glob(os.path.join(segmentation_validation_image_path, "*.png"))
test_files = glob(os.path.join(segmentation_test_image_path, "*.png"))

#train_gen = cxr_generator(BATCH_SIZE,
#                          segmentation_train_path,
#                        'cxrs',
#                        'masks',
#                          target_size=(512, 512))

#val_gen = cxr_generator(BATCH_SIZE,
#                         segmentation_validation_path,
#                         'cxrs',
#                       'masks',
#                         target_size=(512, 512))

test_gen = cxr_generator(BATCH_SIZE,    #generator for the test images,where the "files" is the element in which all the predicted masks will be stored. 
                                        #Uses the cxr_generator function to pass the images into the model for prediction.
                         segmentation_test_path,
                         'cxrs',
                         files = files,
                         target_size=(512, 512))

results = model.predict(test_gen, verbose=1, steps=len(test_files)/BATCH_SIZE)

def save_result(save_path, npyfile, files):  #all pixel values greater than 0.5 after normalisation are turned into 1. Less than 0.5 are turned into 0. So, we get the predicted mask.
    for i, item in enumerate(npyfile):
        result_file = files[i]
        item[item >= 0.50] = 1.0
        item[item < 0.50] = 0.0
        img = (item[:, :, 0] * 255.).astype(np.uint8)
        filename = os.path.basename(result_file)
        result_file = os.path.join(save_path, filename)
        cv2.imwrite(result_file, img)


save_result('/content/predicted_masks', results, files)

Found 39 images belonging to 1 classes.
39/39 [==============================] - 5s 53ms/step


In [ ]:
import os
from glob import glob
import cv2
import numpy as np

true_path = '/content/tbh+clahe/test/masks'
pred_path = '/content/predicted_masks'
true_files = glob(os.path.join(true_path, '*.png'))
pred_files = glob(os.path.join(pred_path, '*.png'))


def diff_mask(ref_image, mask_image):
    mask_image_gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
    mask = cv2.bitwise_and(mask_image, mask_image, mask=mask_image_gray)
    mask_coord = np.where(mask != [0, 0, 0])
    mask[mask_coord[0], mask_coord[1], :] = [255, 0, 0]
    ret = cv2.addWeighted(ref_image, 0.7, mask, 0.3, 0)
    return ret


save_path = '/content/mask_diff'
for true, pred in zip(true_files, pred_files):
    filename = os.path.basename(true)
    ref = cv2.imread(true)
    mask = cv2.imread(pred)
    gg = diff_mask(ref, mask)
    cv2.imwrite(os.path.join(save_path, filename), gg)

segmentation_test_image_path = '/content/tbh+clahe/test/cxrs/'
test_files = glob(os.path.join(segmentation_test_image_path, "*.png"))

seg_path = '/content/segmented_lungs'
for pred, cxr in zip(pred_files, test_files):
    filename = os.path.basename(cxr)
    mask = cv2.imread(pred)
    img = cv2.imread(cxr)
    seg = cv2.bitwise_and(img, mask)
    cv2.imwrite(os.path.join(seg_path, filename), seg)